# 钢条切割

![](img/1_1.png)

$$r_n = max_{0\leq i\leq n}(r_i + r_{n-i})$$

$n$: total length of steel bar

$R$: array saving max gain

$cut$: array saving cutting position

$price$: pre-defined price for each length of steel

In [1]:
n = 10
R = [0]*(n+1)
cut = [0]*(n+1)
price = [0,1,5,8,9,10,17,17,20,24,30]

bottom-up

$i$：length of steel(child problem)

$j$：cutting position (from left)

In [2]:
for i in range(1,n+1):
    for j in range(1,i+1):
        # 这里的R[i-j]一定是提前算过的最优值
        val = price[j] + R[i-j]
        if(val > R[i]):
            R[i] = val
            # 保存长度为i的钢条的切割位置
            cut[i] = j

R[1:]

[1, 5, 8, 10, 13, 17, 18, 22, 25, 30]

backtracking

In [3]:
i = n
while i > 0:
    print(cut[i])
    i -= cut[i]

10


# 最长公共子串(LCS)

In [4]:
X = 'ABCBDAB'
Y = 'BDCABA'
m = len(X)
n = len(Y)
path = []
length = []
for i in range(m+1):
    path.append([0]*(n+1))
    length.append([0]*(n+1))

for i in range(1,m+1):
    for j in range(1,n+1):
        if X[i-1] == Y[j-1]:
            length[i][j] = length[i-1][j-1]+1
            path[i][j] = '↖'
        elif length[i-1][j] >= length[i][j-1]:
            length[i][j] = length[i-1][j]
            path[i][j] = '↑'
        else:
            length[i][j] = length[i][j-1]
            path[i][j] = '←'

length[1:],path[1:]

([[0, 0, 0, 0, 1, 1, 1],
  [0, 1, 1, 1, 1, 2, 2],
  [0, 1, 1, 2, 2, 2, 2],
  [0, 1, 1, 2, 2, 3, 3],
  [0, 1, 2, 2, 2, 3, 3],
  [0, 1, 2, 2, 3, 3, 4],
  [0, 1, 2, 2, 3, 4, 4]],
 [[0, '↑', '↑', '↑', '↖', '←', '↖'],
  [0, '↖', '←', '←', '↑', '↖', '←'],
  [0, '↑', '↑', '↖', '←', '↑', '↑'],
  [0, '↖', '↑', '↑', '↑', '↖', '←'],
  [0, '↑', '↖', '↑', '↑', '↑', '↑'],
  [0, '↑', '↑', '↑', '↖', '↑', '↖'],
  [0, '↖', '↑', '↑', '↑', '↖', '↑']])

重构最优解

In [5]:
def printLCS(X,Y,path,i,j):
    if i == 0 or j == 0:
        return
    if path[i][j] == '↖':
        printLCS(X,Y,path,i-1,j-1)
        print(X[i-1],Y[j-1])
    elif path[i][j] == '↑':
        printLCS(X,Y,path,i-1,j)
    else:
        printLCS(X,Y,path,i,j-1)

printLCS(X,Y,path,m,n)

B B
C C
B B
A A


去掉表path，在$O(1)$的时间内通过length判断path，回溯时注意判断$x[i]$和$y[j]$是否相等

In [6]:
def printLCS(X,Y,length,i,j):
    if i == 0 or j == 0:
        return
    if X[i-1] == Y[j-1] and length[i][j] == length[i-1][j-1]+1:
        printLCS(X,Y,length,i-1,j-1)
        print(X[i-1],end='')
    elif length[i][j] == length[i][j-1]:
        printLCS(X,Y,length,i,j-1)
    elif length[i][j] == length[i-1][j]:
        printLCS(X,Y,length,i-1,j)

print('Length of Longest Common String between %s and %s is %d' % (X,Y,length[m][n]))
print('Longest Common String is: ',end='')
printLCS(X,Y,length,m,n)


Length of Longest Common String between ABCBDAB and BDCABA is 4
Longest Common String is: BDAB

# 矩阵链乘

![](img/1_2.png)

$n$: length of matrix chain **plus 1**

$p[i]$: column number of the $i$th matrix

$Mult[i][j]$：minimum multiplication time of $A_i \cdots A_j$

$Part[i][j]$: optimal backet position of $A_i \cdots A_j$

In [7]:
import math
p = [30,35,15,5,10,20,25]
n = len(p)
Mult = []
Part = []
for i in range(0,n):
    Mult.append([math.inf]*n)
    Part.append([0]*n)

for i in range(n):
    for j in range(0,i+1):
        Mult[i][j] = 0
        
Mult,Part

([[0, inf, inf, inf, inf, inf, inf],
  [0, 0, inf, inf, inf, inf, inf],
  [0, 0, 0, inf, inf, inf, inf],
  [0, 0, 0, 0, inf, inf, inf],
  [0, 0, 0, 0, 0, inf, inf],
  [0, 0, 0, 0, 0, 0, inf],
  [0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0]])

$i$: $i$th matrix

$j$: $j$th matrix

$l$: length of matrix chain (child problem)

- calculating optimal $Mult[i][j]$ requires optimal $M[k][j]$ and $M[i][k]$ where $i \leq k \leq j$, thus we should follow the order of the length of matrix chain, not the ascending order of i and j

In [8]:
# 这是计算了1 1;1 2;1 3;...;1 n；这样算是错的，为什么？
# for i in range(1,n):
#     for j in range(1,n):
#         for k in range(i,j):
#             tmp = Mult[i][k] + Mult[k+1][j] + p[i-1]*p[k]*p[j]
#             if tmp < Mult[i][j]:
#                 Mult[i][j] = tmp

# 计算1 2；2 3；3 4；4 5...
# here n = 7
for l in range(2,n):
    for i in range(1,n-l+1):
        j = i+l-1
        for k in range(i,j):
            tmp = Mult[i][k] + Mult[k+1][j] + p[i-1]*p[k]*p[j]
            if tmp < Mult[i][j]:
                Mult[i][j] = tmp
                Part[i][j] = k
                
Mult[1:],Part[1:]

([[0, 0, 15750, 7875, 9375, 11875, 15125],
  [0, 0, 0, 2625, 4375, 7125, 10500],
  [0, 0, 0, 0, 750, 2500, 5375],
  [0, 0, 0, 0, 0, 1000, 3500],
  [0, 0, 0, 0, 0, 0, 5000],
  [0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 1, 1, 3, 3, 3],
  [0, 0, 0, 2, 3, 3, 3],
  [0, 0, 0, 0, 3, 3, 3],
  [0, 0, 0, 0, 0, 4, 5],
  [0, 0, 0, 0, 0, 0, 5],
  [0, 0, 0, 0, 0, 0, 0]])

Backtracking

In [9]:
def printMatrixChain(Part,i,j):    
    if i == j:
        print('A',end='')
    else:
        print('(',end='')
        k = Part[i][j]
        printMatrixChain(Part,i,k)
        printMatrixChain(Part,k+1,j)
        print(')',end='')

printMatrixChain(Part,1,6)

((A(AA))((AA)A))

In [10]:
X = 'ABCBDAB'
Y = 'BDCABA'
m = len(X)
n = len(Y)
length = []
for i in range(m+1):
    length.append([0]*(n+1))

for i in range(1,m+1):
    for j in range(1,n+1):
        if X[i-1] == Y[j-1]:
            length[i][j] = length[i-1][j-1]+1
        elif length[i-1][j] >= length[i][j-1]:
            length[i][j] = length[i-1][j]
        else:
            length[i][j] = length[i][j-1]

length[1:]

[[0, 0, 0, 0, 1, 1, 1],
 [0, 1, 1, 1, 1, 2, 2],
 [0, 1, 1, 2, 2, 2, 2],
 [0, 1, 1, 2, 2, 3, 3],
 [0, 1, 2, 2, 2, 3, 3],
 [0, 1, 2, 2, 3, 3, 4],
 [0, 1, 2, 2, 3, 4, 4]]

# 最优二叉搜索树

$key[i]$: $i$th key

$p[i]$: searching probability of $i$th key, $0$th is set default to 0

$q[i]$: searching probability of keys between $i$th key and $i+1$th key

$n$: number of keys

$Estimate[i][j]$: Estimiated Cost for the optimal searching binary tree cosisting of keys between $i$ and $j$, endpoints included

$Cost[i][j]$: sum of the probability that points between $i$ and $j$ (endpoints included) is searched

$Root[i][j]$: root of the child tree cosisting of points between $i$ and $j$

In [31]:
import math
key = ['','do','if','read','while']
p = [0,3,3,1,1]
q = [2,3,1,1,1]
n = len(p) - 1
Estimate = []
Cost = []
Root = []

for i in range(n+2):
    Estimate.append([math.inf]*(n+1))
    Cost.append([0]*(n+1))
for i in range(n+1):
    Root.append([0]*(n+1))

# 初始化
for i in range(n+1):
    Estimate[i+1][i] = q[i]
    Cost[i+1][i] = q[i]

n,Estimate,Cost,Root

(4,
 [[inf, inf, inf, inf, inf],
  [2, inf, inf, inf, inf],
  [inf, 3, inf, inf, inf],
  [inf, inf, 1, inf, inf],
  [inf, inf, inf, 1, inf],
  [inf, inf, inf, inf, 1]],
 [[0, 0, 0, 0, 0],
  [2, 0, 0, 0, 0],
  [0, 3, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1]],
 [[0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]])

In [44]:
# l = 1 is not ordinary
for l in range(1,n+1):
    for i in range(1,n-l+2):
        j = i + l
        Cost[i][j-1] = Cost[i][j-2] + p[j-1] + q[j-1]
        for k in range(i,j):
            tmp = Estimate[i][k-1] + Estimate[k+1][j-1] + Cost[i][j-1]
            if tmp < Estimate[i][j-1]:
                Estimate[i][j-1] = tmp
                Root[i][j-1] = k

print('estimation cost of optimal search binary tree is {}'.format(Estimate[1][n]))
Estimate[1:],Cost[1:],Root[1:]

estimation cost of optimal search binary tree is 40


([[2, 13, 25, 32, 40],
  [inf, 3, 11, 17, 25],
  [inf, inf, 1, 5, 11],
  [inf, inf, inf, 1, 5],
  [inf, inf, inf, inf, 1]],
 [[2, 8, 12, 14, 16],
  [0, 3, 7, 9, 11],
  [0, 0, 1, 3, 5],
  [0, 0, 0, 1, 3],
  [0, 0, 0, 0, 1]],
 [[0, 1, 1, 2, 2], [0, 0, 2, 2, 2], [0, 0, 0, 3, 3], [0, 0, 0, 0, 4]])

backtracking: equivalent to ouputing a binary tree

In [42]:
def printBST(Root,i,j,level):
    if i == j+1:
        return
    else:
        r = Root[i][j]
        print('level:{},key:{}'.format(level,key[r]))
        printBST(Root,i,r-1,level+1)
        printBST(Root,r+1,j,level+1)

printBST(Root,1,n,0)

level:0,key:if
level:1,key:do
level:1,key:read
level:2,key:while
